In [1]:
import re
# !pip uninstall spacy
# !pip install nltk
# !pip install spacy==2.3.5
# !pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz
import spacy
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 3000000
import os
from tqdm import tqdm
segment_lengths = [1000, 5000, 10000]

def clean_text(
    string: str, 
    punctuation=r'''!()—-[]{};:'"‘’“”\,<>.…/?@#$%^&*_~''') -> str:

    # Removing the punctuation
    for x in string.lower(): 
        if x in punctuation: 
            string = string.replace(x, "") 

    # Converting the text to lower
    string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split()])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()
    
    # Clean non-alphanumeric characters
    string = re.sub(r'[^a-zA-Z0-9_\s]', '', string)

    return string        

In [2]:
# TEST
os.chdir("../full_novels")
with open('king-1974-Carrie.txt', encoding='utf8') as f:
    contents = f.read()
print(contents[:1000])

 PART ONE 
BLOOD SPORT 

 
News item from the Westover (Me.) weekly Enterprise, August 19, 1966:
 RAIN OF STONES REPORTED 

It was reliably reported by several persons that a rain of stones fell from a clear blue sky on Carlin Street in the town of Chamberlain on August 17th. The stones fell principally on the home of Mrs. Margaret White, damaging the roof extensively and ruining two gutters and a downspout valued at approximately $25. Mrs. White, a widow, lives with her three-year-old daughter, Carietta.
Mrs. White could not be reached for comment.

 Nobody was really surprised when it happened, not really, not at the subconscious level where savage things grow. On the surface, all the girls in the shower room were shocked, thrilled, ashamed, or simply glad that the White bitch had taken it in the mouth again. Some of them might also have claimed surprise, but of course their claim was untrue. Carrie had been going to school with some of them since the first grade, and this had been b

In [3]:
# TEST
clean_text(contents)[:1000]

'part one blood sport news item from the westover me weekly enterprise august 19 1966 rain of stones reported it was reliably reported by several persons that a rain of stones fell from a clear blue sky on carlin street in the town of chamberlain on august 17th the stones fell principally on the home of mrs margaret white damaging the roof extensively and ruining two gutters and a downspout valued at approximately 25 mrs white a widow lives with her threeyearold daughter carietta mrs white could not be reached for comment nobody was really surprised when it happened not really not at the subconscious level where savage things grow on the surface all the girls in the shower room were shocked thrilled ashamed or simply glad that the white bitch had taken it in the mouth again some of them might also have claimed surprise but of course their claim was untrue carrie had been going to school with some of them since the first grade and this had been building since that time building slowly a

# 1. Clean novels (written under real name)

In [4]:
authors, dates, titles, texts = [], [], [], []
for file in tqdm(os.listdir()):
    if (file.endswith(".txt")):
        authors.append(file.split('-')[0])
        dates.append(file.split('-')[1])
        titles.append(file.split('-')[2].split('.')[0])
        with open(file, encoding='utf8') as f:
            contents = f.read()
            doc = nlp(contents, disable=['parser', 'tagger', 'ner', 'lemmatizer'])
            tokenized_contents = ' '.join([token.text for token in doc])
            texts.append(clean_text(tokenized_contents))
print(authors[5], dates[5], titles[5], texts[5][:50])

100%|██████████████████████████████████████████████████████████████████████████████████| 73/73 [08:31<00:00,  7.01s/it]

bachman 1984 Thinner chapter 1 246 thinner the old gypsy man with the r


In [6]:
import pandas as pd
books_df = pd.DataFrame({'author': authors,
                    'date':dates, 
                       'title':titles,
                      'text':texts})
books_df

,author,date,title,text
0,bachman,1966.67,The_Long_Walk,part one starting out the long walk chapter 1 ...
1,bachman,1968.69,Roadwork,prologue i do nt know why you do nt know why m...
2,bachman,1970,Rage,chapter 01 the morning i got it on was nice a ...
3,bachman,1973,Blaze,chapter 1 george was somewhere in the dark bla...
4,bachman,1981.2,The_Running_Man,minus 100 and counting she was squinting at th...
...,...,...,...,...
59,straub,1993,The_Throat,part one tim underhill 1 an alcoholic homicide...
60,straub,1995,Hellfire_Club,book i before dawn in a time just before this ...
61,straub,1999,Mr_X,1 how i came home and why 1 stupid me i fell r...
62,straub,2003,Lost_Boy_Lost_Girl,the dead mother part one 1 nancy underhill s d...


# 2. Get descriptive statistics - wordcount, unique tokens, TTR (first 10,000 tokens and from full texts)

In [7]:
def wordcount(text):
    return len(text.split(' '))
books_df['wordcount'] = books_df.text.apply(wordcount)
books_df

,author,date,title,text,wordcount
0,bachman,1966.67,The_Long_Walk,part one starting out the long walk chapter 1 ...,87333
1,bachman,1968.69,Roadwork,prologue i do nt know why you do nt know why m...,93047
2,bachman,1970,Rage,chapter 01 the morning i got it on was nice a ...,55909
3,bachman,1973,Blaze,chapter 1 george was somewhere in the dark bla...,82444
4,bachman,1981.2,The_Running_Man,minus 100 and counting she was squinting at th...,67769
...,...,...,...,...,...
59,straub,1993,The_Throat,part one tim underhill 1 an alcoholic homicide...,254781
60,straub,1995,Hellfire_Club,book i before dawn in a time just before this ...,191789
61,straub,1999,Mr_X,1 how i came home and why 1 stupid me i fell r...,186987
62,straub,2003,Lost_Boy_Lost_Girl,the dead mother part one 1 nancy underhill s d...,89152


In [8]:
from collections import Counter
def ttr(text, full_text=True):
    if full_text:
        tokenlist = text.split(' ')
    else: 
        tokenlist = text.split(' ')[:10000]
    types = Counter(tokenlist)
    return len(types)/len(tokenlist)

def types(text):
    return len(Counter(text.split(' ')))

In [9]:
books_df['TTR_10k'] = books_df.text.apply(ttr, full_text=False)
books_df['TTR_full'] = books_df.text.apply(ttr)
books_df

,author,date,title,text,wordcount,TTR_10k,TTR_full
0,bachman,1966.67,The_Long_Walk,part one starting out the long walk chapter 1 ...,87333,0.1965,0.090779
1,bachman,1968.69,Roadwork,prologue i do nt know why you do nt know why m...,93047,0.2088,0.094425
2,bachman,1970,Rage,chapter 01 the morning i got it on was nice a ...,55909,0.2090,0.110948
3,bachman,1973,Blaze,chapter 1 george was somewhere in the dark bla...,82444,0.1873,0.093639
4,bachman,1981.2,The_Running_Man,minus 100 and counting she was squinting at th...,67769,0.2520,0.126592
...,...,...,...,...,...,...,...
59,straub,1993,The_Throat,part one tim underhill 1 an alcoholic homicide...,254781,0.2179,0.049592
60,straub,1995,Hellfire_Club,book i before dawn in a time just before this ...,191789,0.2315,0.061317
61,straub,1999,Mr_X,1 how i came home and why 1 stupid me i fell r...,186987,0.2435,0.069577
62,straub,2003,Lost_Boy_Lost_Girl,the dead mother part one 1 nancy underhill s d...,89152,0.2444,0.090228


In [10]:
books_df['unique_tokens'] = books_df.text.apply(types)
books_df

,author,date,title,text,wordcount,TTR_10k,TTR_full,unique_tokens
0,bachman,1966.67,The_Long_Walk,part one starting out the long walk chapter 1 ...,87333,0.1965,0.090779,7928
1,bachman,1968.69,Roadwork,prologue i do nt know why you do nt know why m...,93047,0.2088,0.094425,8786
2,bachman,1970,Rage,chapter 01 the morning i got it on was nice a ...,55909,0.2090,0.110948,6203
3,bachman,1973,Blaze,chapter 1 george was somewhere in the dark bla...,82444,0.1873,0.093639,7720
4,bachman,1981.2,The_Running_Man,minus 100 and counting she was squinting at th...,67769,0.2520,0.126592,8579
...,...,...,...,...,...,...,...,...
59,straub,1993,The_Throat,part one tim underhill 1 an alcoholic homicide...,254781,0.2179,0.049592,12635
60,straub,1995,Hellfire_Club,book i before dawn in a time just before this ...,191789,0.2315,0.061317,11760
61,straub,1999,Mr_X,1 how i came home and why 1 stupid me i fell r...,186987,0.2435,0.069577,13010
62,straub,2003,Lost_Boy_Lost_Girl,the dead mother part one 1 nancy underhill s d...,89152,0.2444,0.090228,8044


In [11]:
books_df[['author', 'date', 'title', 'wordcount', 'unique_tokens', 'TTR_10k', "TTR_full"]].to_csv('descriptive_stats_books.csv')

# 4. Divide cleaned texts into chunks 1,000, 5,000, and 10,000-token chunks and save

In [12]:
def get_chunks(text, windowsize):
    tokenlist = text.split()
    chunks = []
    for i in range(0, len(tokenlist), windowsize):
        if len(tokenlist[i:i + windowsize]) == windowsize: # do not include trailing words after the the last whole segment
            chunks.append(" ".join(tokenlist[i:i + windowsize]))
    return chunks

In [13]:
for segment_length in segment_lengths: 
    directory = "segments/{}token_segments".format(segment_length)
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    for i in range(books_df.shape[0]):
        chunks = get_chunks(books_df.loc[i]['text'], segment_length)

        # Get the chunk and its position in the list of chunks
        for ii, chunk in tqdm(enumerate(chunks)):
            print(len(chunk.split()))
            #Label each new file with the name of the book, and the segment number
            with open('{}/{}-{}-{}-{}.txt'.format(directory, 
                                                              books_df.loc[i]['author'],
                                                               books_df.loc[i]['date'],
                                                               books_df.loc[i]['title'], 
                                                                ii), 
                                                              'w', encoding='utf8') as f:
                f.write(chunk)
                print(books_df.loc[i]['author'])
                print(books_df.loc[i]['title'])

87it [00:00, 751.18it/s]
0it [00:00, ?it/s]

1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1000
bachman
The_Long_Walk
1

93it [00:00, 661.25it/s]
55it [00:00, 812.04it/s]
0it [00:00, ?it/s]

1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachman
Roadwork
1000
bachm

82it [00:00, 679.97it/s]
0it [00:00, ?it/s]

...

straub
Floating_Dragon
10000
straub
Floating_Dragon
10000
straub
Floating_Dragon
10000
straub
Floating_Dragon
10000
straub
Floating_Dragon
10000
straub
Floating_Dragon
10000
straub
Floating_Dragon
10000
straub
Floating_Dragon
10000
straub
Floating_Dragon
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Koko
10000
straub
Mystery
10000
straub
Mystery
10000
straub
Mystery


18it [00:00, 277.53it/s]
25it [00:00, 301.81it/s]
0it [00:00, ?it/s]

10000
straub
Mystery
10000
straub
Mystery
10000
straub
Mystery
10000
straub
Mystery
10000
straub
Mystery
10000
straub
Mystery
10000
straub
Mystery
10000
straub
Mystery
10000
straub
Mystery
10000
straub
Mystery
10000
straub
Mystery
10000
straub
Mystery
10000
straub
Mystery
10000
straub
Mystery
10000
straub
Mystery
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
The_Throat
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club


19it [00:00, 513.89it/s]
18it [00:00, 521.44it/s]
8it [00:00, 914.79it/s]
10it [00:00, 365.25it/s]

10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Hellfire_Club
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Mr_X
10000
straub
Lost_Boy_Lost_Girl
10000
straub
Lost_Boy_Lost_Girl
10000
straub
Lost_Boy_Lost_Girl
10000
straub
Lost_Boy_Lost_Girl
10000
straub
Lost_Boy_Lost_Girl
10000
straub
Lost_Boy_Lost_Girl
10000
straub
Lost_Boy_Los